# Logistic Regression

The likelihood we are trying to maximize is:

$$ L(\beta) = \prod_{i=1}^n p(x_i^T \beta)^{y_i} (1 - p(x_i^T \beta))^{1 - y_i}, $$

where $p(z) = \frac{1}{1 + e^{-z}}$ is the logistic function. In other words, $\log( \frac{p(z)}{1 - p(z)} ) = z$. Taking logs, we obtain:

\begin{align*}
\log L(\beta) = \sum_{i=1}^n y_i \log p(x_i^T \beta) + (1 - y_i) \log(1 - p(x_i^T \beta)) &= \sum_{i=1}^n y_i \log \frac{p(x_i^T \beta)}{1 - p(x_i^T \beta)} + \log(1 - p(x_i^T \beta)) \\
&= \sum_{i=1}^n y_i x_i^T \beta + \log(1 - p(x_i^T \beta)) \\
&= \sum_{i=1}^n y_i x_i^T \beta + \log(1 + e^{x_i^T \beta})
\end{align*}

Now we take the derivative with respect to $\beta$ to get the gradient.

\begin{align*}
\frac{\partial}{\partial\beta} \log L(\beta) &= \sum_{i=1}^n x_i y_i - x_i \frac{e^{x_i^T \beta}}{1 + e^{x_i^T \beta}} \\
&= \sum_{i=1}^n x_i (y_i - p(x_i^T \beta)) \\
&= X^T (y - p(X\beta)).
\end{align*}

And we take the derivative with respect to $\beta$ again to get the Hessian matrix.

\begin{align*}
\frac{\partial^2}{\partial\beta^2} \log L(\beta) &= -X^T \underbrace{\textrm{diag}\left\{p(x_i^T \beta) (1 - p(x_i^T \beta))\right\}}_{W} X
\end{align*}

Now we consider Newton's method, an iterative method for finding the maximum of this function:

\begin{align*} 
\beta^{(t+1)} &\leftarrow \beta^{(t)} - \left[ \frac{\partial^2}{\partial\beta^2} \log L(\beta^{(t)}) \right]^{-1} \frac{\partial}{\partial\beta} \log L(\beta^{(t)}) \\
&= \beta^{(t)} + (X^T W X)^{-1} X^T (y - p(X\beta^{(t)}))
\end{align*}


In [1]:
import numpy as np

In [2]:
def logistic(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def logistic_regression(X, y):
    beta = np.zeros(X.shape[1])
    for _ in range(5):
        p = logistic(np.dot(X, beta))
        w = p * (1 - p)
        beta += np.linalg.solve(np.dot(X.T, (w[:, np.newaxis] * X)), np.dot(X.T, y - p))
        
    p = logistic(np.dot(X, beta))
    w = p * (1 - p)
    se = np.sqrt(np.diag(np.linalg.inv(np.dot(X.T, (w[:, np.newaxis] * X)))))
    
    return beta, se

In [4]:
# generate some data
X = np.random.randn(1000, 100)
y = 1 * (np.random.randn(1000) > 0)

In [5]:
# use our logistic regression function
beta, se = logistic_regression(X, y)

# calculate the z-score for each coefficient
z = beta / se
z

array([ -1.17822066e+00,   5.72936614e-01,  -1.16279084e+00,
        -1.79970941e+00,  -7.90654108e-01,   7.14672399e-01,
        -4.15126802e-01,   1.54652856e+00,  -1.31705830e+00,
         5.47136796e-01,  -9.48387775e-01,   1.84822450e-01,
         1.09421346e+00,  -7.53202161e-01,   1.20919011e+00,
        -1.31714282e-01,  -1.69463959e+00,   1.87930432e+00,
         4.54353207e-02,  -2.29460844e-01,  -9.70618977e-01,
         3.71460355e-01,   1.25516126e+00,  -1.17435087e+00,
         8.93476654e-02,   7.84153263e-02,  -3.72888080e-01,
         1.86502665e+00,  -6.91869561e-01,   4.56499978e-01,
        -1.34347286e+00,   6.79682847e-01,   8.43489422e-04,
        -1.50485100e+00,   5.64174696e-01,   9.66234639e-01,
         7.15517890e-01,   2.54577878e+00,  -4.05769622e-02,
         1.86931933e-03,   1.16062667e-01,  -6.79503942e-01,
         8.46867171e-01,   7.02812572e-01,  -7.65949194e-01,
        -1.00929555e+00,   8.64718794e-01,   9.85437203e-01,
        -1.37998708e+00,

In [6]:
# compare the z-scores we got above with statsmodels
from statsmodels.api import Logit
model = Logit(y, X)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.635935
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      900
Method:                           MLE   Df Model:                           99
Date:                Wed, 31 May 2017   Pseudo R-squ.:                 0.08252
Time:                        20:00:32   Log-Likelihood:                -635.93
converged:                       True   LL-Null:                       -693.13
                                        LLR p-value:                    0.1382
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0812      0.069     -1.178      0.239      -0.216       0.054
x2             0.0397      0.069      0.573      0.567      -0.096       0.176
x3            -0.0846      0.073     -1.163      0.245      -0.227       0.058
x4            -0.1301      0.072     -1.800      0.072      -0.272       0.012
x5            -0.0560      0.071     -0.791      0.429      -0.195       0.083
x6             0.0501      0.070      0.715      0.475      -0.087       0.187
x7            -0.0295      0.071     -0.415      0.678      -0.169       0.110
x8             0.1101      0.071      1.547      0.122      -0.029       0.250
x9            -0.0932      0.071     -1.317      0.188      -0.232       0.046
x10            0.0382      0.070      0.547      0.584      -0.099       0.175
x11           -0.0698      0.074     -0.948      0.343      -0.214       0.074
x12            0.0128      0.070      0.185      0.853      -0.123       0.149
x13            0.0815      0.074      1.094      0.274      -0.064       0.227
x14           -0.0547      0.073     -0.753      0.451      -0.197       0.088
x15            0.0891      0.074      1.209      0.227      -0.055       0.234
x16           -0.0089      0.067     -0.132      0.895      -0.141       0.123
x17           -0.1171      0.069     -1.695      0.090      -0.253       0.018
x18            0.1304      0.069      1.879      0.060      -0.006       0.266
x19            0.0032      0.071      0.045      0.964      -0.135       0.141
x20           -0.0169      0.074     -0.229      0.819      -0.161       0.127
x21           -0.0691      0.071     -0.971      0.332      -0.209       0.070
x22            0.0264      0.071      0.371      0.710      -0.113       0.166
x23            0.0844      0.067      1.255      0.209      -0.047       0.216
x24           -0.0833      0.071     -1.174      0.240      -0.222       0.056
x25            0.0066      0.074      0.089      0.929      -0.139       0.152
x26            0.0055      0.071      0.078      0.937      -0.133       0.144
x27           -0.0271      0.073     -0.373      0.709      -0.169       0.115
x28            0.1373      0.074      1.865      0.062      -0.007       0.282
x29           -0.0474      0.069     -0.692      0.489      -0.182       0.087
x30            0.0321      0.070      0.456      0.648      -0.106       0.170
x31           -0.0942      0.070     -1.343      0.179      -0.232       0.043
x32            0.0470      0.069      0.680      0.497      -0.089       0.183
x33         5.697e-05      0.068      0.001      0.999      -0.132       0.132
x34           -0.1026      0.068     -1.505      0.132      -0.236       0.031
x35            0.0397      0.070      0.564      0.573      -0.098       0.177
x36            0.0673      0.070      0.966      0.334      -0.069       0.204
x37            0.0508      0.071      0.716      0.474      -0.088       0.190
x38            0.1920      0.075      2.546      0.011       0.044       0.340
x3